In [7]:
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
# %run ConV_VAE.ipynb
# %run bigger_latent_space.ipynb
%run rgb200x200.ipynb
%run loader.ipynb
%run test.ipynb
%run train.ipynb

In [8]:
plt.rcParams['figure.dpi'] = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
####### IMPORTANT ######
# set this flag to true if you want to load the model beforehand
load = False
# specify which model you want to load
model_name = None

latent_space = 512
batch_size=64

In [9]:
from torchvision import datasets

normalize = transforms.Lambda(lambda x : x / 255.0)
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])
# mnist_data = datasets.MNIST('./data',
#              transform=transforms.ToTensor(),
#             download=True)
# train_data = datasets.FashionMNIST('./data/RGB_ASL/asl_alphabet_train',
#                             transform=transforms.ToTensor(),
#                             download=True)
train_data = datasets.ImageFolder('./data/RGB_ASL/asl_alphabet_train/asl_alphabet_train/', transform=transforms.ToTensor())
# test_data = datasets.ImageFolder('./data/RGB_ASL/asl_alphabet_test/asl_alphabet_test/', transform=transforms.ToTensor())
# train_data = CustomImageDataset('../data/sign_mnist_train.csv', transform=transform)
# test_data = CustomImageDataset('../data/sign_mnist_test.csv', transform=transform)

In [10]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [11]:
model = cVAE((3,200,200), 29).to(device)
if (load):
    model.load_state_dict(torch.load(model_name, map_location=device))

criterion = torch.nn.BCELoss(reduction="sum")
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)

OutOfMemoryError: CUDA out of memory. Tried to allocate 134.00 MiB (GPU 0; 4.00 GiB total capacity; 3.49 GiB already allocated; 0 bytes free; 3.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
epochs = 5
train_losses = []
test_losses = []
for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    # Train on data
    train_loss = train_vae(train_loader, model, optimizer, device)
    #test_loss = test(epoch, model, train_loader)
    print('====> Average Training loss per image: {:.4f}'.format(train_loss.item()))
    print('====> Average  Test set loss per image : {:.4f}'.format(test_loss))
    if epoch % 5 == 0:
        torch.save(model.state_dict(), "../model/model" + str(epoch) + ".pt")
        with torch.no_grad():
            sample = torch.randn(batch_size, latent_space).to(device)
            sample = model.decoder(sample).cpu()
            save_image(sample.view(batch_size, 1, 28, 28),
                       '../results/' + str(epoch) + '.png')
    train_losses.append(train_loss.cpu().detach().numpy())
    #test_losses.append(test_loss)

In [ ]:
plt.plot(train_losses, label="training")
#plt.plot(test_losses, label="validation")
plt.legend()
plt.show()